In [ ]:
from imageanalysis.making_stacks import *
dir1 = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/'
dir2 = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2525/26-03-2025/'
# process_directory_h5(dir1, output_file='different_sides.h5',sort_by='time')
process_directory_h5(dir2, output_file='same_sides.h5',sort_by='time')

In [ ]:
%matplotlib qt5
font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 

different_sides_irradiation = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/different_sides.h5'
same_sides_irradiation = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2525/26-03-2025/same_sides.h5'

from imageanalysis.image_processing import CalibratedImages
calibrated_images = CalibratedImages(same_sides_irradiation, font_path=font_path)

In [ ]:
%matplotlib qt5
font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 

different_sides_irradiation = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/different_sides.h5'

same_sides_irradiation = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2525/26-03-2025/same_sides.h5'
from imageanalysis.feature_analysis import FeaturesAnalysis


features_analysis = FeaturesAnalysis(different_sides_irradiation, font_path=font_path)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

datafile = 'Sample_2476/cluster_density_analysis/2l_gr.csv'
datafile = 'Sample_2525/cluster_density_analysis/4l_gr.csv'
datafile = 'analysis_results.csv'
df = pd.read_csv(datafile)
df['Roundness']
df.head()

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import ast
from itertools import chain
import numpy as np
from scipy.signal import savgol_filter
from scipy.stats import norm, beta, gamma, lognorm, gaussian_kde
def get_shape_metric_info(sh_metric):
    """Converts string-encoded list column into flat numeric list."""
    name = sh_metric.name
    shape_metric = sh_metric.apply(ast.literal_eval)
    values = list(chain.from_iterable(shape_metric))
    return values, name

def plot_shape_metric_distribution(sh_metric, name, n_layers=None, as_line=False):
    """Plot single shape metric as histogram or line curve."""
    plt.figure(figsize=(10, 6))
    
    if as_line:
        counts, bins = np.histogram(sh_metric, bins=50)
        centers = (bins[:-1] + bins[1:]) / 2
        plt.plot(centers, counts, lw=2, color='blue')
    else:
        plt.hist(sh_metric, bins=50, density=True, alpha=0.8, color='blue')

    title = f'{name.replace("_", " ").title()} Distribution'
    if n_layers:
        title += f' ({n_layers} Layers)'
    plt.title(title)
    plt.xlabel(name.replace("_", " ").title())
    plt.ylabel('Density')
    plt.grid()
    plt.show()


def plot_shape_metric_comparison(metric_list, n_layers=None, as_line=False, smooth=True, window_length=15, polyorder=5):
    """Plot histogram or smoothed line for multiple shape metrics."""
    fig, ax = plt.subplots(1, len(metric_list), figsize=(5 * len(metric_list), 5))
    if len(metric_list) == 1:
        ax = [ax]

    for i, (sh_metric, name) in enumerate(metric_list):
        counts, bins = np.histogram(sh_metric, bins=100, density=False)
        centers = (bins[:-1] + bins[1:]) / 2
        kde = gaussian_kde(sh_metric)
        pdf = kde(centers)
        factor = np.sum(counts * pdf) / np.sum(pdf ** 2)
        pdf *= factor  # make the PDF to match histogram area
        #  mean and std for Gaussian fit
        dx = centers[1] - centers[0]
        pdf_norm  = pdf / np.sum(pdf) / dx  # normalize to area=1
        mean_pdf = np.sum(centers * pdf_norm) * dx
        std_pdf = np.sqrt(np.sum((centers - mean_pdf) ** 2 * pdf_norm) * dx)
        if as_line:
            if smooth:
                # Ensure window length is valid
                wl = min(window_length, len(centers) - 1 if len(centers) % 2 == 0 else len(centers))
                if wl % 2 == 0:
                    wl -= 1
                if wl >= 5:
                    counts_smooth = savgol_filter(counts, window_length=wl, polyorder=polyorder)
                else:
                    counts_smooth = counts  # fallback if too short
                ax[i].plot(centers, counts_smooth, lw=2, color='blue', label='Smoothed')
            else:
                ax[i].plot(centers, counts, lw=2, color='blue', label='Raw Line')
        else:
            ax[i].hist(sh_metric, bins=100, alpha=0.8, color='blue', label='Histogram')

        # mu, std = norm.fit(sh_metric)
        # gaussian = norm.pdf(centers, mu, std)
        # factor = np.sum(counts * gaussian) / np.sum(gaussian **2)
        # gaussian *= factor  


        title = f'{name.replace("_", " ").title()}'
        if n_layers:
            title += f' ({n_layers} Layers)'
        ax[i].plot(centers, pdf, 'r--', label=f'Gaussian Fit\nμ = {mean_pdf:.4f}, σ = {std_pdf:.4f}')
        ax[i].set_title(title)
        ax[i].set_xlabel(name.replace("_", " ").title())
        ax[i].set_ylabel('Density')
        ax[i].legend()
        ax[i].grid()

    if n_layers:
        plt.suptitle(f"Shape Metrics Comparison for {n_layers} Layers", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()



def plot_hist_with_gaussian_fit(data, label, bins=100, smooth=True):
    data = np.array(data)
    mu, std = norm.fit(data)

    # Histogram
    counts, bin_edges = np.histogram(data, bins=bins, density=True)
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    if smooth:
        y_fit = savgol_filter(counts, window_length=11, polyorder=3)
    else:
        y_fit = counts

    # Gaussian curve for comparison
    x_fit = np.linspace(min(data), max(data), 1000)
    gaussian = norm.pdf(x_fit, mu, std)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.plot(bin_centers, y_fit, label='Data (smoothed)', lw=2)
    plt.plot(x_fit, gaussian, 'r--', label=f'Gaussian Fit\nμ = {mu:.4f}, σ = {std:.4f}')
    plt.xlabel(label)
    plt.ylabel('Density')
    plt.title(f'{label} Distribution with Gaussian Fit')
    plt.legend()
    plt.grid(True)
    plt.show()

    return mu, std


# Main analysis
datafile = 'Sample_2476/cluster_density_analysis/{}l_gr.csv'
i = [2, 3, 4, 5]

for number_of_layers in i:
    df = pd.read_csv(datafile.format(number_of_layers))
    
    circularities, name1     = get_shape_metric_info(df['Circularities'])
    feret_diameters, name2   = get_shape_metric_info(df['Feret_Diameter'])
    aspect_ratios, name3     = get_shape_metric_info(df['Aspect_Ratio'])
    clusters_area, name4     = get_shape_metric_info(df['Clusters'])
    mean_circularity = np.array(circularities).mean()
    mean_feret_diameter = np.array(feret_diameters).mean()
    mean_aspect_ratio = np.array(aspect_ratios).mean()
    mean_clusters_area = np.array(clusters_area).mean()
    print(f"Mean Circularity for {number_of_layers} layers: {mean_circularity:.4f}")
    print(f"Mean Feret Diameter for {number_of_layers} layers: {mean_feret_diameter:.4f}")
    print(f"Mean Aspect Ratio for {number_of_layers} layers: {mean_aspect_ratio:.4f}")
    print(f"Mean Clusters Area for {number_of_layers} layers: {mean_clusters_area:.4f}")



    plot_shape_metric_comparison([
        (circularities, name1),
        (feret_diameters, name2),
        (aspect_ratios, name3),
        (clusters_area, name4),
    ], n_layers=number_of_layers, as_line=False)